In [1]:
from gensim import utils, corpora, models, similarities
from simserver import SessionServer
import pandas as pd

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
#server = SessionServer('/Users/lindsay/Documents/Data Science/Insight/Project/simserver')
server = SessionServer('simserver/')

In [2]:
# load pre-processed text
preproc_text = pd.read_pickle('gensim/preprocessed_text_v2.pkl')

In [3]:
preproc_text.head()

,id,language,text
0,16392,English,"[host, review, favorit, favorit, classic, pro,..."
0,18489,English,"[everyon, need, hope, everyon, need, encourag,..."
0,12350,English,"[insur, news, interview, rate, announc, insur,..."
0,12351,English,"[amovetv, crew, talk, video, game, esport, lot..."
0,12352,English,"[pastor, greg, lauri, thirtyminut, daili, radi..."


In [4]:
corpus = [{'id' : int(x['id']),
           'tokens' : x['text']}
         for ind, x in preproc_text.iterrows()]

In [5]:
# upload corpus to server
utils.upload_chunked(server, corpus, chunksize=1000)

In [6]:
# train server (uses LSI and default dimensionality of 400 over a TF-IDF representation)
server.train(corpus, method='lsi')

ERROR:sqlitedict:failed to delete /var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldictaaaad
Traceback (most recent call last):
  File "//anaconda/lib/python2.7/site-packages/sqlitedict.py", line 283, in terminate
    os.remove(self.filename)
OSError: [Errno 2] No such file or directory: '/var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldictaaaad'
ERROR:sqlitedict:failed to delete /var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldict27aa32
Traceback (most recent call last):
  File "//anaconda/lib/python2.7/site-packages/sqlitedict.py", line 283, in terminate
    os.remove(self.filename)
OSError: [Errno 2] No such file or directory: '/var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldict27aa32'
ERROR:sqlitedict:failed to delete /var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldict9cf695
Traceback (most recent call last):
  File "//anaconda/lib/python2.7/site-packages/sqlitedict.py", line 283, in terminate
    os.remove(self.filename)
OSError: [Errno 2] No such file or dire

In [7]:
# index documents we trained on
server.index(corpus)

ERROR:sqlitedict:failed to delete /var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldict3283db
Traceback (most recent call last):
  File "//anaconda/lib/python2.7/site-packages/sqlitedict.py", line 283, in terminate
    os.remove(self.filename)
OSError: [Errno 2] No such file or directory: '/var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldict3283db'
ERROR:sqlitedict:failed to delete /var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldictec1b65
Traceback (most recent call last):
  File "//anaconda/lib/python2.7/site-packages/sqlitedict.py", line 283, in terminate
    os.remove(self.filename)
OSError: [Errno 2] No such file or directory: '/var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldictec1b65'
ERROR:sqlitedict:failed to delete /var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldictcd8a95
Traceback (most recent call last):
  File "//anaconda/lib/python2.7/site-packages/sqlitedict.py", line 283, in terminate
    os.remove(self.filename)
OSError: [Errno 2] No such file or di

In [8]:
# query for similar documents
sim_result = server.find_similar('18183', max_results=25)
sim_result

[('18183', 1.0000001192092896, None),
 ('12691', 0.9298461675643921, None),
 ('18414', 0.8706507086753845, None),
 ('18412', 0.845548689365387, None),
 ('17351', 0.8358715176582336, None),
 ('16392', 0.8268610835075378, None),
 ('15734', 0.8127365708351135, None),
 ('13911', 0.8093626499176025, None),
 ('14625', 0.8076587915420532, None),
 ('18413', 0.8020862936973572, None),
 ('14913', 0.771027684211731, None),
 ('16789', 0.7585797309875488, None),
 ('17528', 0.740717351436615, None),
 ('16449', 0.7386183142662048, None),
 ('16766', 0.7303283214569092, None),
 ('17246', 0.7225890755653381, None),
 ('12600', 0.704534113407135, None),
 ('13309', 0.6827985644340515, None),
 ('17778', 0.5448945760726929, None),
 ('15052', 0.5416157841682434, None),
 ('17787', 0.5352489948272705, None),
 ('12897', 0.47698116302490234, None),
 ('18257', 0.4752431809902191, None),
 ('13440', 0.4672621488571167, None),
 ('12534', 0.45265477895736694, None)]

In [31]:
# query for text not in corpus
import preprocess_text
doc = {'tokens': preprocess_text.preprocess_text('cats and recipes and politics')}
sim_result = server.find_similar(doc)

In [32]:
ids = [x[0] for x in sim_result]

In [45]:
def pg_int_array(the_list):
    return '(' + ','.join(the_list) + ')'

In [36]:
id_array = list_to_pg_arr(ids)

In [46]:
import psycopg2
dbname = 'podcast'
username = 'lindsay'
con = psycopg2.connect(database = dbname, user = username)
cursor = con.cursor()
query = "SELECT name FROM podcast WHERE id IN %s;"
cursor.execute(query % (list_to_pg_arr(ids)))
cursor.fetchall()

[('A Way with Words',),
 ('AboutBeverages.com - Podcast',),
 ('Alive Again - Pet Reincarnation on Pet Life Radio (PetLifeRadio.com)',),
 ('All in the Industry',),
 ("America's Test Kitchen Radio",),
 ('Animal Instinct',),
 ('Animal Writes - Animal Writers and Best-selling Authors - Pets & Animals on Pet Life Radio (PetLifeRadio.com)',),
 ('Answer Me This!',),
 ('Awesome Advocates- For Dogs, Cats, & other Pets on Pet Life Radio (PetLifeRadio.com)',),
 ('Bark And Swagger - Pet Fashion on Pet Life Radio (PetLifeRadio.com)',),
 ('Bartender Journey - Cocktails. Spirits. Bartending Culture. Libations for your Ears.',),
 ('Bear Friend Tea Party',),
 ('itunes \xe2\x80\x93 Big Cat Rescue',),
 ('Brains On',),
 ('Cat and Fox',),
 ('Cat Context Podcast',),
 ("Chef's Story",),
 ('Comedy Factory from CBC Radio',),
 ('comedy4cast - Comedy Podcast',),
 ('Cooking Issues',),
 ('Cooking on the Wildside Vodcast',),
 ('Cooking with Paula McIntyre',),
 ('Deep House Cat',),
 ('Doctor Who: Mostly Harmless Cut

In [51]:
podcast_id=15793

In [55]:
query = "SELECT name FROM podcast WHERE id = '%s'"
cursor.execute(query % podcast_id)
podcast_name = cursor.fetchall()

In [61]:
podcast_name[0][0].decode('utf-8')

u'Meowster Podcast'